In [6]:
from dotenv import load_dotenv
from typing import Optional
import os
import yaml
import pandas as pd
import vectorbt as vbt
import numpy as np

load_dotenv()
config_path = os.getenv('3CANDLES_CONFIG_PATH')

with open(config_path, 'r') as file:
    config = yaml.safe_load(file)
    
df_hour = pd.read_csv(config['Data_filename_hour'])
df_hour = df_hour.set_index('Time')
df_hour.index = pd.to_datetime(df_hour.index)

df_min = pd.read_csv(config['Data_filename_minute'])
df_min = df_min.set_index('Time')
df_min.index = pd.to_datetime(df_min.index)

df_hour = df_hour.drop(columns=['Volume', 'High', 'Low'])
df_hour['Dir'] = 0

df_hour.loc[df_hour['Close'] > df_hour['Open'], 'Dir'] = 1
bull_entry_mask = ((df_hour['Dir'] == 1) & (df_hour['Dir'].shift(1) == 1) & (df_hour['Dir'].shift(2) == 1) & #short
                    (df_hour['Close'].shift(2) < df_hour['Open']))
df_hour['Bull Entry'] = bull_entry_mask
bear_entry_mask = ((df_hour['Dir'] == 0) & (df_hour['Dir'].shift(1) == 0) & (df_hour['Dir'].shift(2) == 0) & #long
                    (df_hour['Close'].shift(2) > df_hour['Open']))
df_hour['Bear Entry'] = bear_entry_mask

df_hour.loc[df_hour['Bull Entry'] == True, 'SL'] = df_hour['Close'] + ((df_hour['Close'] - df_hour['Close'].shift(2)) * (config['RR'] * 0.5)) #short
df_hour.loc[df_hour['Bear Entry'] == True, 'SL'] = df_hour['Close'] - ((df_hour['Close'].shift(2) - df_hour['Close']) * (config['RR'] * 0.5))#long

df_hour.loc[df_hour['Bull Entry'] == True, 'TP'] = df_hour['Close'] - ((df_hour['Close'] - df_hour['Close'].shift(2)) * config['RR']) #short
df_hour.loc[df_hour['Bear Entry'] == True, 'TP'] = df_hour['Close'] + ((df_hour['Close'].shift(2) - df_hour['Close']) * config['RR']) #long

df_min['Bull Entry'] = df_hour['Bull Entry'].shift(1).reindex(df_min.index, method='ffill')
df_min['Bear Entry'] = df_hour['Bear Entry'].shift(1).reindex(df_min.index, method='ffill')
df_min['SL'] = df_hour['SL'].shift(1).reindex(df_min.index, method='ffill')
df_min['TP'] = df_hour['TP'].shift(1).reindex(df_min.index, method='ffill')
df_min['Date and hour'] = df_min.index.floor('h')
df_min.dropna(inplace=True)

In [7]:
index_arr_min = df_min.index.to_numpy()
open_arr_min = df_min['Open'].to_numpy()
close_arr_min = df_min['Close'].to_numpy()
bull_entry_arr_min = df_min['Bull Entry'].to_numpy()
bear_entry_arr_min = df_min['Bear Entry'].to_numpy()
sl_arr_min = df_min['SL'].to_numpy()
tp_arr_min = df_min['TP'].to_numpy()
date_and_hour_arr_min = df_min['Date and hour'].to_numpy()
price_arr_min = np.full(len(index_arr_min), np.nan)
bull_entrymask_arr_min = np.full(len(index_arr_min), False)
bear_entrymask_arr_min = np.full(len(index_arr_min), False)
bull_exit_arr_min = np.full(len(index_arr_min), False)
bear_exit_arr_min = np.full(len(index_arr_min), False)

In [8]:
trade_direct: Optional[str] = None
cur_sl: Optional[float] = None; cur_tp: Optional[float] = None
opened_date_and_hour = {}

for i in range(len(index_arr_min)):
    if trade_direct is None:
        date_and_hour = date_and_hour_arr_min[i]

        if bull_entry_arr_min[i] and opened_date_and_hour.get(date_and_hour) == None:
            trade_direct = 'bull'
            bull_entrymask_arr_min[i] = True
            cur_sl, cur_tp = sl_arr_min[i], tp_arr_min[i]
            price_arr_min[i] = close_arr_min[i]
            opened_date_and_hour[date_and_hour] = True
        elif bear_entry_arr_min[i] and opened_date_and_hour.get(date_and_hour) == None:
            trade_direct = 'bear'
            bear_entrymask_arr_min[i] = True
            cur_sl, cur_tp = sl_arr_min[i], tp_arr_min[i]
            price_arr_min[i] = close_arr_min[i]
            opened_date_and_hour[date_and_hour] = True
        else:
            continue  

    elif trade_direct == 'bull':

        if close_arr_min[i] >= cur_sl:
            price_arr_min[i] = cur_sl
            trade_direct, cur_sl, cur_tp = None, None, None
            bull_exit_arr_min[i] = True
        elif close_arr_min[i] <= cur_tp:
            price_arr_min[i] = cur_tp
            trade_direct, cur_sl, cur_tp = None, None, None
            bull_exit_arr_min[i] = True

    elif trade_direct == 'bear':

        if close_arr_min[i] <= cur_sl:
            price_arr_min[i] = cur_sl
            trade_direct, cur_sl, cur_tp = None, None, None
            bear_exit_arr_min[i] = True
        elif close_arr_min[i] >= cur_tp:
            price_arr_min[i] = cur_tp
            trade_direct, cur_sl, cur_tp = None, None, None
            bear_exit_arr_min[i] = True

pf = vbt.Portfolio.from_signals(
entries = bear_entrymask_arr_min,
exits = bear_exit_arr_min,
short_entries = bull_entrymask_arr_min,
short_exits = bull_exit_arr_min,
price = price_arr_min,
open = df_min["Open"],
close = df_min["Close"],
size = config['Trade']['size'],
size_type = config['Trade']['size_type'],
fees = config['Broker']['fees'],
fixed_fees = config['Broker']['fixed_fees'],
slippage = config['Slippage'],
init_cash = config['Initial_cash'],
freq = '1m'
)

In [9]:
pf.stats()

Start                         2024-01-02 12:00:00+00:00
End                           2025-12-05 17:59:00+00:00
Period                                112 days 15:20:00
Start Value                                     50000.0
End Value                                  51596.298645
Total Return [%]                               3.192597
Benchmark Return [%]                           6.153594
Max Gross Exposure [%]                        30.428761
Total Fees Paid                                     0.0
Max Drawdown [%]                                1.92177
Max Drawdown Duration                  23 days 15:00:00
Total Trades                                        610
Total Closed Trades                                 609
Total Open Trades                                     1
Open Trade PnL                                14.499109
Win Rate [%]                                  39.573071
Best Trade [%]                                 2.347911
Worst Trade [%]                                -

In [ ]:
pf.plot().show()

In [10]:
pf.trades.records_readable.tail(21)

,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
589,589,0,13309.570018,2025-11-28 05:01:00+00:00,1.15891,0.0,2025-11-28 09:00:00+00:00,1.15787,0.0,13.841953,0.000897,Short,Closed,589
590,590,0,13337.897555,2025-11-28 09:01:00+00:00,1.15676,0.0,2025-11-28 10:55:00+00:00,1.15564,0.0,-14.938445,-0.000968,Long,Closed,590
591,591,0,13347.408136,2025-11-28 10:56:00+00:00,1.15560,0.0,2025-11-28 11:16:00+00:00,1.15580,0.0,2.669482,0.000173,Long,Closed,591
592,592,0,13346.137801,2025-11-28 11:17:00+00:00,1.15577,0.0,2025-11-28 16:00:00+00:00,1.15816,0.0,31.897269,0.002068,Long,Closed,592
593,593,0,13300.789248,2025-11-28 16:01:00+00:00,1.16043,0.0,2025-12-01 12:47:00+00:00,1.16400,0.0,-47.483818,-0.003076,Short,Closed,593
594,594,0,13245.254095,2025-12-01 12:48:00+00:00,1.16422,0.0,2025-12-01 12:49:00+00:00,1.16334,0.0,11.655824,0.000756,Short,Closed,594
595,595,0,13240.751381,2025-12-01 13:00:00+00:00,1.16488,0.0,2025-12-02 00:17:00+00:00,1.16064,0.0,56.140786,0.003640,Short,Closed,595
596,596,0,13303.289225,2025-12-02 00:18:00+00:00,1.16067,0.0,2025-12-02 01:16:00+00:00,1.16044,0.0,-3.059757,-0.000198,Long,Closed,596
597,597,0,13304.676322,2025-12-02 01:17:00+00:00,1.16048,0.0,2025-12-02 04:01:00+00:00,1.16120,0.0,9.579367,0.000620,Long,Closed,597
598,598,0,13298.443550,2025-12-02 04:02:00+00:00,1.16124,0.0,2025-12-02 11:00:00+00:00,1.16062,0.0,8.245035,0.000534,Short,Closed,598
